In [2]:
import pandas as pd
df_geo=pd.read_csv("olist_geolocation_dataset.csv",encoding='utf-8')
df_geo

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP
...,...,...,...,...,...
1000158,99950,-28.068639,-52.010705,tapejara,RS
1000159,99900,-27.877125,-52.224882,getulio vargas,RS
1000160,99950,-28.071855,-52.014716,tapejara,RS
1000161,99980,-28.388932,-51.846871,david canabarro,RS


In [2]:
cities=df_geo['geolocation_city'].unique().tolist()
print(cities)

['sao paulo', 'são paulo', 'sao bernardo do campo', 'jundiaí', 'taboão da serra', 'sãopaulo', 'sp', 'sa£o paulo', 'sao jose dos campos', 'osasco', 'carapicuíba', 'carapicuiba', 'barueri', 'santana de parnaiba', 'pirapora do bom jesus', 'santana de parnaíba', 'jandira', 'itapevi', 'cotia', 'taboao da serra', 'vargem grande paulista', 'embu das artes', 'itapecerica da serra', 'embu', 'são lourenço da serra', 'sao lourenco da serra', 'embu-guacu', 'embu-guaçu', 'embu guaçu', 'juquitiba', 'embu guacu', 'embuguacu', 'guarulhos', 'adamantina', 'guarulhos-sp', 'aruja', 'arujá', 'santa isabel', 'mairipora', 'mairiporã', 'cajamar', 'caieiras', 'jordanesia', 'polvilho', 'mauá', 'jordanésia', 'franco da rocha', 'francisco morato', 'poa', 'itaquaquecetuba', 'ferraz de vasconcelos', 'poá', 'suzano', 'mogi das cruzes', 'mogidascruzes', 'salesopolis', 'biritiba-mirim', 'guararema', 'salesópolis', 'biritiba mirim', 'santo andre', 'santo andré', 'maua', 'ribeirão pires', 'ribeirao pires', 'rio grande d

In [4]:
df_sellers=pd.read_csv('olist_sellers_dataset.csv')
df_sellers

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP
...,...,...,...,...
3090,98dddbc4601dd4443ca174359b237166,87111,sarandi,PR
3091,f8201cab383e484733266d1906e2fdfa,88137,palhoca,SC
3092,74871d19219c7d518d0090283e03c137,4650,sao paulo,SP
3093,e603cf3fec55f8697c9059638d6c8eb5,96080,pelotas,RS


In [9]:
state_city_dict=df_sellers.groupby('seller_state')['seller_city'].unique().apply(list).to_dict()
state_city_dict

{'AC': ['rio branco'],
 'AM': ['manaus'],
 'BA': ['lauro de freitas',
  'porto seguro',
  'salvador',
  'barro alto',
  'bahia',
  "arraial d'ajuda (porto seguro)",
  'irece',
  'ipira',
  'eunapolis',
  'ilheus',
  'guanambi',
  'feira de santana'],
 'CE': ['fortaleza',
  'mucambo',
  'pacatuba',
  'juzeiro do norte',
  'eusebio',
  'caucaia',
  'varzea alegre'],
 'DF': ['brasilia', 'brasilia df', 'gama'],
 'ES': ['viana',
  'cachoeiro de itapemirim',
  'cariacica / es',
  'vila velha',
  'domingos martins',
  'vitoria',
  'cariacica',
  'colatina',
  'afonso claudio',
  'serra',
  'muqui'],
 'GO': ['anapolis',
  'santa terezinha de goias',
  'uruacu',
  'goiania',
  'piracanjuba',
  'formosa',
  'luziania',
  'morrinhos',
  'abadia de goias',
  'rio verde',
  'jaragua',
  'aparecida de goiania'],
 'MA': ['sao luis'],
 'MG': ['belo horizonte',
  'camanducaia',
  'congonhal',
  'araguari',
  'contagem',
  'tres coracoes',
  'guiricema',
  'claudio',
  'ponte nova',
  'caratinga',
  'ub

In [15]:
from rapidfuzz import process

# Step 1: Define cleaning function
def clean_city_name(row):
    state = row['geolocation_state']
    messy_city = row['geolocation_city']
    
    # Get valid cities for this state from sellers dataset
    valid_cities = state_city_dict.get(state)
    if not valid_cities:
        return messy_city  # No cities to compare with
    
    # Fuzzy match
    best_match, score, _ = process.extractOne(messy_city, valid_cities)
    return best_match if score >= 85 else messy_city

# Step 2: Apply cleaning
df_geo['geolocation_city_cleaned'] = df_geo.apply(clean_city_name, axis=1)

# Step 3: Get unique cleaned cities
cities2 = df_geo['geolocation_city_cleaned'].unique().tolist()
print(cities2)

['sao paulo', 'sao bernardo do campo', 'são paulo', 'jundiai', 'taboao da serra', 'sãopaulo', 'sp', 'sao jose dos campos', 'osasco', 'carapicuiba', 'barueri', 'santana de parnaiba', 'caxias do sul', 'jandira', 'itapevi', 'cotia', 'vargem grande paulista', 'embu das artes', 'itapecerica da serra', 'serra negra', 'embu guacu', 'embu-guaçu', 'juquitiba', 'guarulhos', 'adamantina', 'aruja', 'arujá', 'santa barbara d´oeste', 'mairipora', 'cajamar', 'caieiras', 'jordanesia', 'polvilho', 'mauá', 'jordanésia', 'franco da rocha', 'francisco morato', 'poa', 'itaquaquecetuba', 'ferraz de vasconcelos', 'poá', 'suzano', 'mogi das cruzes', 'salesopolis', 'biritiba-mirim', 'guara', 'salesópolis', 'biritiba mirim', 'santo andre', 'maua', 'ribeirao pires', 'rio claro', 'sao caetano do sul', 'sbc', 'diadema', 'santos', 'bertioga', 'caruara', 'sao vicente', 'guaruja', 'cubatao', 'cubatão', 'sao sebastiao', 'ubatuba', 'ilhabela', 'caraguatatuba', 'são sebastião', 'maresias', 'praia grande', 'mongagua', 'm

In [19]:
import pandas as pd
from rapidfuzz import process

# Step 1: Normalize both datasets to lowercase and strip spaces
df_geo['geolocation_city'] = df_geo['geolocation_city'].str.lower().str.strip()
df_geo['geolocation_state'] = df_geo['geolocation_state'].str.upper()

df_sellers['seller_city'] = df_sellers['seller_city'].str.lower().str.strip()
df_sellers['seller_state'] = df_sellers['seller_state'].str.upper()

# Step 2: Build state-wise city dictionary from sellers_df
state_city_dict = df_sellers.groupby('seller_state')['seller_city'].unique().to_dict()

# Step 3: Define fuzzy matching function
def clean_city_name(row):
    state = row['geolocation_state']
    messy_city = row['geolocation_city']
    
    valid_cities = state_city_dict.get(state)
    if valid_cities is None or len(valid_cities) == 0:
        return messy_city  # No valid cities to match with

    result = process.extractOne(messy_city, valid_cities)
    
    if result is not None:
        best_match, score, _ = result
        if score >= 80:
            return best_match

    return messy_city



# Step 4: Apply to DataFrame
df_geo['geolocation_city_cleaned'] = df_geo.apply(clean_city_name, axis=1)

# Step 5: Optional - See what was changed
df_changes = df_geo[df_geo['geolocation_city'] != df_geo['geolocation_city_cleaned']][['geolocation_city', 'geolocation_city_cleaned']]
print(df_changes.sample(10))


               geolocation_city                  geolocation_city_cleaned
365165                andradina                                  andradas
474745                  niterói                                   niteroi
396625  espirito santo do turvo                               santo andre
990034                cruz alta                         santa cruz do sul
4836                  são paulo                                 sao paulo
887786        cornélio procópio                         cornelio procopio
969816           capao da canoa                 santo antonio da patrulha
820850                   cuiabá                                    cuiaba
955779          sapucaia do sul  novo hamburgo, rio grande do sul, brasil
472483                  niterói                                   niteroi


In [20]:
cities3 = df_geo['geolocation_city_cleaned'].unique().tolist()
print(cities3)

['sao paulo', 'sao bernardo do campo', 'são paulo', 'jundiai', 'taboao da serra', 'sp', 'sao jose dos campos', 'osasco', 'carapicuiba', 'barueri', 'santana de parnaiba', 'caxias do sul', 'jandira', 'itapevi', 'cotia', 'vargem grande paulista', 'embu das artes', 'itapecerica da serra', 'serra negra', 'embu guacu', 'juquitiba', 'guarulhos', 'adamantina', 'aruja', 'santa barbara d´oeste', 'mairipora', 'cajamar', 'caieiras', 'jordanesia', 'polvilho', 'mauá', 'jordanésia', 'franco da rocha', 'francisco morato', 'poa', 'itaquaquecetuba', 'ferraz de vasconcelos', 'poá', 'suzano', 'mogi das cruzes', 'salesopolis', 'biritiba-mirim', 'guara', 'salesópolis', 'biritiba mirim', 'santo andre', 'maua', 'ribeirao pires', 'rio claro', 'sao caetano do sul', 'sbc', 'diadema', 'santos', 'bertioga', 'caruara', 'sao vicente', 'guaruja', 'cubatao', 'cubatão', 'sao sebastiao', 'ubatuba', 'ilhabela', 'caraguatatuba', 'maresias', 'praia grande', 'mongagua', 'mongaguá', 'itanhaem', 'peruibe', 'itariri', 'sao ped

In [22]:
import pandas as pd
from rapidfuzz import process

# Step 1: Normalize both datasets to lowercase and strip spaces
df_geo['geolocation_city'] = df_geo['geolocation_city'].str.lower().str.strip()
df_geo['geolocation_state'] = df_geo['geolocation_state'].str.upper()

df_cust=pd.read_csv('olist_customers_dataset.csv')
df_cust['customer_city'] = df_cust['customer_city'].str.lower().str.strip()
df_cust['customer_state'] = df_cust['customer_state'].str.upper()

# Step 2: Build state-wise city dictionary from sellers_df
state_city_dict = df_cust.groupby('customer_state')['customer_city'].unique().to_dict()

# Step 3: Define fuzzy matching function
def clean_city_name(row):
    state = row['geolocation_state']
    messy_city = row['geolocation_city']
    
    valid_cities = state_city_dict.get(state)
    if valid_cities is None or len(valid_cities) == 0:
        return messy_city  # No valid cities to match with

    result = process.extractOne(messy_city, valid_cities)
    
    if result is not None:
        best_match, score, _ = result
        if score >= 80:
            return best_match

    return messy_city



# Step 4: Apply to DataFrame
df_geo['geolocation_city_cleaned'] = df_geo.apply(clean_city_name, axis=1)

# Step 5: Optional - See what was changed
df_changes = df_geo[df_geo['geolocation_city'] != df_geo['geolocation_city_cleaned']][['geolocation_city', 'geolocation_city_cleaned']]
print(df_changes.sample(10))


               geolocation_city geolocation_city_cleaned
272176      são josé dos campos      sao jose dos campos
904406            florianópolis            florianopolis
139951                são paulo                sao paulo
131604                são paulo                sao paulo
666437            teófilo otoni            teofilo otoni
270183      são josé dos campos      sao jose dos campos
119356                são paulo                sao paulo
684805             santanópolis                  santana
708103  amparo de sao francisco            sao francisco
334047              altinópolis              altinopolis


In [23]:
cities4 = df_geo['geolocation_city_cleaned'].unique().tolist()
print(cities4)

['sao paulo', 'sao bernardo do campo', 'sao paulo do potengi', 'jundiai', 'taboao da serra', 'espigao', 'sao jose dos campos', 'osasco', 'carapicuiba', 'barueri', 'santana de parnaiba', 'pirapora do bom jesus', 'jandira', 'itapevi', 'cotia', 'vargem grande paulista', 'embu das artes', 'itapecerica da serra', 'embu', 'sao lourenco da serra', 'embu-guacu', 'juquitiba', 'guarulhos', 'adamantina', 'aruja', 'santa isabel', 'mairipora', 'cajamar', 'caieiras', 'jordanesia', 'polvilho', 'mauá', 'jordanésia', 'franco da rocha', 'francisco morato', 'poa', 'itaquaquecetuba', 'ferraz de vasconcelos', 'poá', 'suzano', 'mogi das cruzes', 'salesopolis', 'biritiba-mirim', 'guararema', 'santo andre', 'maua', 'ribeirao pires', 'rio grande da serra', 'sao caetano do sul', 'sbcampo', 'diadema', 'santos', 'bertioga', 'caruara', 'sao vicente', 'guaruja', 'cubatao', 'sao sebastiao', 'ubatuba', 'ilhabela', 'caraguatatuba', 'areias', 'sao francisco', 'praia grande', 'mongagua', 'itanhaem', 'peruibe', 'itariri'

In [24]:
state_city_dict1=df_cust.groupby('customer_state')['customer_city'].unique().apply(list).to_dict()
state_city_dict1

{'AC': ['rio branco',
  'senador guiomard',
  'cruzeiro do sul',
  'porto acre',
  'epitaciolandia',
  'xapuri',
  'brasileia',
  'manoel urbano'],
 'AL': ['maceio',
  'arapiraca',
  'cajueiro',
  'canapi',
  'palmeira dos indios',
  'sao jose da tapera',
  'penedo',
  'agua branca',
  'teotonio vilela',
  'colonia leopoldina',
  'murici',
  'campo alegre',
  'santana do mundau',
  'sao sebastiao',
  'sao luis do quitunde',
  'girau do ponciano',
  'rio largo',
  'mar vermelho',
  'belem',
  'maragogi',
  'santana do ipanema',
  'pilar',
  'porto calvo',
  'satuba',
  'junqueiro',
  'barra de sao miguel',
  "tanque d'arca",
  'paulo jacinto',
  'monteiropolis',
  'taquarana',
  'piacabucu',
  "olho d'agua das flores",
  'lagoa da canoa',
  'paripueira',
  'delmiro gouveia',
  'santa luzia do norte',
  'maravilha',
  'batalha',
  'dois riachos',
  'pindoba',
  'maribondo',
  'igreja nova',
  'barra de santo antonio',
  'ibateguara',
  'boca da mata',
  'igaci',
  'inhapi',
  'traipu',
 

In [25]:
state_city_dict2=df_geo.groupby('geolocation_state')['geolocation_city_cleaned'].unique().apply(list).to_dict()
state_city_dict2

{'AC': ['sao paulo',
  'rio de janeiro',
  'sena madureira',
  'rio branco',
  'feijo',
  'senador guiomard',
  'cruzeiro do sul',
  'xapuri',
  'feijó',
  'manoel urbano',
  'santa rosa do purus',
  'placido de castro',
  'tarauaca',
  'assis brasil',
  'mâncio lima',
  'mancio lima',
  'tarauacá',
  'porto walter',
  'brasileia',
  'campinas',
  'rodrigues alves',
  'acrelandia',
  'plácido de castro',
  'epitaciolandia',
  'bujari',
  'marechal thaumaturgo',
  'porto acre',
  'capixaba',
  'jordão',
  'jordao',
  'acrelândia'],
 'AL': ['maceio',
  'maceia³',
  'barra de sao miguel',
  'rio largo',
  'marechal deodoro',
  'pilar',
  'satuba',
  'santa luzia do norte',
  'coqueiro seco',
  'coruripe',
  'sao sebastiao',
  'feliz deserto',
  'teotonio vilela',
  'junqueiro',
  'penedo',
  'igreja nova',
  'sao miguel dos campos',
  'luziapolis',
  'piacabucu',
  'jequia da praia',
  'roteiro',
  'campo alegre',
  'piaçabuçu',
  'porto calvo',
  'poxim',
  'anadia',
  'arapiraca',
  'fe

In [1]:
df_rev=pd.read_csv('olist_order_reviews_dataset.csv')
df_rev

NameError: name 'pd' is not defined

In [6]:
import pandas as pd
from googletrans import Translator
import time

trans=Translator()
def trans_text(text,idx):
    try:
        if pd.isna(text):
            return text
        time.sleep(1)
        return trans.translate(str(text),src='pt',dest='en').text
    except Exception as e:
        print(f"Error translating row:{idx}:'{text}'->{e}")
        return text

df_rev['english_review_comment_title']=[trans_text(text,idx) for idx,text in enumerate(df_rev['review_comment_title'])]
df_rev['english_review_comment_message']=[trans_text(text,idx) for idx,text in enumerate(df_rev['review_comment_message'])]

df_rev

Error translating row:512:'Otimo'->_ssl.c:989: The handshake operation timed out
Error translating row:662:'Super recomendo'->the JSON object must be str, bytes or bytearray, not NoneType
Error translating row:994:'Produto entregue'->The read operation timed out
Error translating row:2371:'Recomendo!!'->The read operation timed out
Error translating row:3013:'Parceiro não confiável '->The read operation timed out
Error translating row:4506:'Bom '->The read operation timed out
Error translating row:5912:'Recomendo!!!'->The read operation timed out
Error translating row:6439:'Excelente!!!'->The read operation timed out
Error translating row:6913:'Não recomendo'->The read operation timed out
Error translating row:7556:'pessimo'->_ssl.c:989: The handshake operation timed out
Error translating row:7623:'Satisfatório '->_ssl.c:989: The handshake operation timed out
Error translating row:10081:'Otimo produto'->The read operation timed out
Error translating row:11130:'Mercadoria não foi entreg

KeyboardInterrupt: 

In [ ]:
import pandas as pd
from deep_translator import GoogleTranslator
import time

# Function to translate safely
def translate_text(text):
    try:
        if pd.isna(text):
            return text
        return GoogleTranslator(source='pt', target='en').translate(text)
    except Exception as e:
        print(f"Error translating row : '{text}' -> {e}")
        return text  # return original if translation fails

# Translate specific columns
df_rev['english_review_comment_message'] = [translate_text(text) for text in df_rev['review_comment_message']]

df_rev


Error translating row : 'Produto dentro das expectativas e entrega super rapida. ' -> Produto dentro das expectativas e entrega super rapida. --> No translation was found using the current translator. Try another translator?
Error translating row : 'Produto muito bonito ' -> Produto muito bonito --> No translation was found using the current translator. Try another translator?
Error translating row : 'ddddd' -> ddddd --> No translation was found using the current translator. Try another translator?
Error translating row : 'Gostei chegou no prazo sem burocracia' -> Gostei chegou no prazo sem burocracia --> No translation was found using the current translator. Try another translator?
Error translating row : 'meu filho amou otimo produto' -> meu filho amou otimo produto --> No translation was found using the current translator. Try another translator?
Error translating row : 'Comprei dois produtos chegou um só ,no envio fala que foi postado 2 , chegou somente 1 quero meu produto ou meu d

In [ ]:
df_rev.to_csv('translated_reviews.csv', index=False)

In [ ]:
df_rev['english_review_comment_title'] = [translate_text(text) for text in df_rev['review_comment_title']]

In [3]:
input_file = "olist_order_reviews_dataset.csv"
output_file = "olist_order_reviews_dataset_utf8.csv"

with open(input_file, "r", encoding="windows-1252", errors="ignore") as src, \
     open(output_file, "w", encoding="utf-8") as dest:
    contents = src.read()
    dest.write(contents)

print("✅ File converted to UTF-8 (bad characters ignored).")



✅ File converted to UTF-8 (bad characters ignored).


In [1]:
input_path = 'C:\\Users\\Ruthveek\\Downloads\\olist_order_reviews_dataset_u.csv'
output_path = 'C:\\Users\\Ruthveek\\Downloads\\olist_order_reviews_dataset_utf8_cleaned.csv'

with open(input_path, 'r', encoding='windows-1252', errors='replace') as infile:
    content = infile.read()

with open(output_path, 'w', encoding='utf-8') as outfile:
    outfile.write(content)

print(f"✅ Cleaned file saved to: {output_path}")


✅ Cleaned file saved to: C:\Users\Ruthveek\Downloads\olist_order_reviews_dataset_utf8_cleaned.csv
